## VHF Analysis

This notebook will walkthrough plotting and analysis for water models.  The raw data is in TXT format and contains the van hove function for the various water models.  For more information on how to compute the van hove function, please see [MDTraj](https://www.mdtraj.org/1.9.5/index.html) and the [scattering](https://github.com/mattwthompson/scattering) code.

### Importing packages
Start off by importing the analysis module within this package

In [ ]:
from water_vhf_analysis.analysis import plot_relaxation
from water_vhf_analysis.analysis import plot_summary
from water_vhf_analysis.analysis import plot_partials
from water_vhf_analysis.utils.utils import get_txt_file, get_csv_file
import numpy as np

### Loading in raw data
The raw data is expected to be loaded in as a Python dictionary with the following keys: "Name", "r", "t", and "g"

In [ ]:
# Load in SPC/E data
spce = {
     "r": np.loadtxt(get_txt_file("spce/overlap_nvt", "r_final.txt")),
     "t": np.loadtxt(get_txt_file("spce/overlap_nvt", "t_final.txt")),
     "g": np.loadtxt(get_txt_file("spce/overlap_nvt", "vhf_final.txt")),
     "name": "SPC/E (CMD)",
 }

# Load in TIP3P_EW data
tip3pew = {
     "r": np.loadtxt(get_txt_file("tip3p_ew/overlap_nvt", "r_final.txt")),
     "t": np.loadtxt(get_txt_file("tip3p_ew/overlap_nvt", "t_final.txt")),
     "g": np.loadtxt(get_txt_file("tip3p_ew/overlap_nvt", "vhf_final.txt")),
     "name": "TIP3P_EW (CMD)",
 }

# Load in reaxff data
bk3 = {
     "r": np.loadtxt(get_txt_file("bk3/overlap_nvt", "r_final.txt")),
     "t": np.loadtxt(get_txt_file("bk3/overlap_nvt", "t_final.txt")),
     "g": np.loadtxt(get_txt_file("bk3/overlap_nvt", "vhf_final.txt")),
     "name": "BK3 (Polarizable CMD)",
 }

# Load in DFTB data
dftb = {
     "r": np.loadtxt(get_txt_file("dftb/overlap_nvt", "r_final.txt")),
     "t": np.loadtxt(get_txt_file("dftb/overlap_nvt", "t_final.txt")),
     "g": np.loadtxt(get_txt_file("dftb/overlap_nvt", "vhf_final.txt")),
     "name": "3obw (DFTB)",
 }

# Load in reaxff data
reaxff = {
     "r": np.loadtxt(get_txt_file("reaxff/overlap_nvt", "r_final.txt")),
     "t": np.loadtxt(get_txt_file("reaxff/overlap_nvt", "t_final.txt")),
     "g": np.loadtxt(get_txt_file("reaxff/overlap_nvt", "vhf_final.txt")),
     "name": "CHON-2017_weak (ReaxFF)",
 }

# Load in aimd data
aimd = {
     "r": np.loadtxt(get_txt_file("aimd/overlap_nvt", "r_final.txt")),
     "t": np.loadtxt(get_txt_file("aimd/overlap_nvt", "t_final.txt")),
     "g": np.loadtxt(get_txt_file("aimd/overlap_nvt", "vhf_final.txt")),
     "name": "optB88 (AIMD)",
 }

# Load in aimd data at 300K
aimd_330 = {
     "r": np.loadtxt(get_txt_file("aimd/330k/overlap_nvt", "r_final.txt")),
     "t": np.loadtxt(get_txt_file("aimd/330k/overlap_nvt", "t_final.txt")),
     "g": np.loadtxt(get_txt_file("aimd/330k/overlap_nvt", "vhf_final.txt")),
     "name": "optB88 at 330 K (AIMD)",
 }

# Load in IXS data
# Multiplying r by 0.1 to convert from angstroms to nm
# Adding 1 to g(r,t)-1
ixs = {
     "r": 0.1 * np.loadtxt(get_txt_file("expt", "R_1811pure.txt"))[0],
     "t": np.loadtxt(get_txt_file("expt", "t_1811pure.txt"))[:,0],
     "g": 1 + np.loadtxt(get_txt_file("expt", "VHF_1811pure.txt")),
     "name": "IXS",
 }

data = [ixs, spce, tip3pew, bk3, reaxff, dftb, aimd, aimd_330]

### Plotting Total VHF

A subplot containing $G(r, t)$ as a function of distance and a heatmap of $G(r, t)-1$ can be plotted with the function `plot_total_subplots`.  The first argument is our list of dictionaries containing the raw data, and the `save` argument is set to False since we don't want to save out this figure.

In [ ]:
plot_summary.plot_total_subplots(data, save=False)

### Plotting correlations
The functions to plot correlations are contained within `plot_relaxation.py`.  In the paper, the correlations related to the first neighbor peak are analyzed through the magnitude of G(r, t) as well as through the area under the curve, A(t), of the first peak.  This is plotted with the function `plot_first_peak_subplot()`.

In [ ]:
# Plot the correlations for the first peak
plot_relaxation.plot_first_peak_subplot(data, save=False)

A(t) displays a two-step decay.  We can fit this decay to an exponential equation.  We can perform this curve fitting and plot the results for each model.

In [ ]:
# Plot the fits for A(t)
plot_relaxation.first_peak_auc(data, save=False)

When running this function, the fitting results are saved to a CSV file.  This can be loaded into a Pandas DataFrame to view.

In [ ]:
import pandas as pd

df = pd.read_csv(get_csv_file("first_peak_fits.csv"))
df

The height of the second peak, $G_2(t)$, was also calculated.  Due to different values at $t=0$, these values were also normalized.

In [ ]:
# Plot the correlations for the second peak
plot_relaxation.plot_second_subplot(data, save=False)

### Plot partial Van Hove functions
The van hove functions for the specific pairwise interactions were also computed.  These functions are contained within `plot_partials`.  For these functions, I'm going to use a function to load in the data.

In [ ]:
# Load in oxygen-oxygen partial data

oo = plot_partials.get_data("O_O")
plot_partials.plot_vhf_subplots(oo)

In [ ]:
# Load in oxygen-hydrogen partial data

oh = plot_partials.get_data("O_H")
plot_partials.plot_vhf_subplots(oh)

In [ ]:
# Load in hydrogen-hydrogen partial data

hh = plot_partials.get_data("H_H")
plot_partials.plot_vhf_subplots(hh)

The height of the partial peaks have also been plotted.

In [ ]:
# Plot correlations for partial van hove functions

plot_partials.plot_peak_subplots(save=False)